- Source: https://tableau.github.io/server-client-python/docs/api-ref
- "HDC현대산업개발": "2a21240d-3f53-4f02-944d-b01647c3dd51"

In [1]:
import tableauserverclient as TSC
import pandas as pd

In [2]:
hr = pd.read_excel("D:/디지털혁신팀/태블로/협업플랫폼 사용량/협업플랫폼 사용량_FINAL/HR/직원명단_202106.xlsx")
hr["성명"] = hr["성명"].apply(lambda x:x.replace(" ", ""))

In [3]:
empls = {str(row["사번"]):(row["본부명"], row["조직명"], row["직무"], row["성명"]) for _, row in hr.iterrows()}

req_opts = TSC.RequestOptions(pagesize=1000)

## SIgn In

In [157]:
with open("D:/디지털혁신팀/태블로/admin_TOKEN.pkl", "rb") as f:
    token = pk.load(f)
tableau_auth = TSC.PersonalAccessTokenAuth("admin_TOKEN", token)
server = TSC.Server("http://218.153.56.75/", use_server_version=True)

## Projects

In [48]:
with server.auth.sign_in(tableau_auth):
    all_project_items, pagination_item = server.projects.get(req_options=req_opts)
    print([(prj.name, prj.id) for prj in all_project_items])

[('기본값', '7ff34228-4f13-11eb-8d94-cbbd9e107787'), ('Tableau 샘플', '07b1c1a3-d2fb-4a12-b4ba-c65b396f593c'), ('HDC현대산업개발', '2a21240d-3f53-4f02-944d-b01647c3dd51'), ('6월 데이터 확인', '44546321-94c8-4746-b0e5-0dfb3fa4ce5a')]


## Workbooks

In [307]:
# with server.auth.sign_in(tableau_auth):
#     all_wbs, pagination_item = server.workbooks.get()
# all_wbs = [(wb.project_name, wb.name) for wb in all_wbs]
# all_wbs = pd.DataFrame(all_wbs, columns=["prj", "wb"])
# all_wbs = all_wbs.sort_values(by=["prj", "wb"])

## Groups

## groups.get(), groups.populate_users()

In [161]:
with server.auth.sign_in(tableau_auth):
    all_groups, pagination_item = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])
all_groups = all_groups.sort_values(by=["group", "group_id"])
all_groups = all_groups.reset_index(drop=True)

In [162]:
all_groups.to_csv("D:/디지털혁신팀/태블로/all_groups.csv", index=False, encoding="euc-kr")
all_groups.head()

,group,group_id
0,"""HR현황"" 접근 가능 그룹",8bae0a19-c83c-4713-b932-a489754653de
1,"""본부장 회의"" 접근 가능 그룹",01b579e9-e0b0-458d-b12a-94292171d3a7
2,"""분양일보"" 접근 가능 그룹",16065e45-21b3-4e1b-80e1-c7c41079adfd
3,All Users,80307210-4f13-11eb-8d95-5bc411fd9c23
4,디지털혁신팀,516aa22e-b1ef-47b1-ad64-6e5116622c7f


In [145]:
group_user = list()
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    for group in groups[1:]:
        pagination_item = server.groups.populate_users(group, req_options=req_opts)
        group_user.extend([(group.name,) + empls[user.name] + (user.site_role,) if user.name in empls.keys() else (group.name, None, None, None, user.name, user.site_role) for user in group.users])

group_user = pd.DataFrame(group_user, columns=["group", "hq", "dep", "job", "name", "site_role"])
group_user = group_user.sort_values(by=["group", "hq", "dep", "job", "name", "site_role"])
group_user = group_user.reset_index(drop=True)

In [146]:
group_user.to_csv("D:/디지털혁신팀/태블로/group_user.csv", index=False, encoding="euc-kr")
group_user.head()

,group,hq,dep,job,name,site_role
0,"""HR현황"" 접근 가능 그룹",건설본부,건설기획팀,팀원,심창식,Viewer
1,"""HR현황"" 접근 가능 그룹",미래혁신본부,HR혁신팀,팀장,선영욱,Viewer
2,"""HR현황"" 접근 가능 그룹",미래혁신본부,디지털플랫폼팀,팀원,박미숙,Viewer
3,"""HR현황"" 접근 가능 그룹",미래혁신본부,미래전략팀,서무,김아현,Viewer
4,"""본부장 회의"" 접근 가능 그룹",개발영업본부,도시개발1팀,팀원,임유빈,Viewer


### groups.create()

In [66]:
try:
    with server.auth.sign_in(tableau_auth):
        newgroup = TSC.GroupItem("마이호미")
        newgroup.minimum_site_role = TSC.UserItem.Roles.Viewer
        newgroup = server.groups.create(newgroup)
except:
    pass

### groups.add_user()

In [140]:
all_users.loc[all_users["name"]=="1101296", "user_id"]

146    06296de2-b463-4cff-afe4-24fda0cc5967
Name: user_id, dtype: object

In [141]:
with server.auth.sign_in(tableau_auth):
    server.groups.add_user(groups[-1], "06296de2-b463-4cff-afe4-24fda0cc5967")

## Users

### users.get(), users.populate_groups()

In [126]:
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get(req_options=req_opts)
all_users = [empls[user.name] + (user.site_role, user.id) if user.name in empls.keys() else (None, None, None, user.name, user.site_role, user.id) for user in all_users]
all_users = pd.DataFrame(all_users, columns=["hq", "dep", "job", "name", "site_role", "user_id"])
all_users = all_users.sort_values(by=["hq", "dep", "job", "name", "site_role", "user_id"])
all_users = all_users.reset_index(drop=True)

In [127]:
all_users.to_csv("D:/디지털혁신팀/태블로/all_users.csv", index=False, encoding="euc-kr")
all_users.head()

,hq,dep,job,name,site_role,user_id
0,개발영업본부,개발부문,부문장,김태수,Viewer,1f34b28f-5859-4d04-ad50-b17fb4cc1061
1,개발영업본부,개발영업본부,본부장,박희윤,Viewer,aeac9fd0-7f64-4888-9a70-9374508927f8
2,개발영업본부,견적그룹(개발영업),그룹장,소문석,Viewer,6f6243e2-7d03-4238-a055-8e28178efaa3
3,개발영업본부,도시개발1팀,팀원,임유빈,Viewer,9ceb0a5f-5875-4e0b-b951-a0d56bdeea30
4,개발영업본부,도시개발1팀,팀장,이현우,Viewer,2c06e1d5-14b6-4afe-8c26-4a6fe8e9e8b8


In [28]:
user_group = list()
with server.auth.sign_in(tableau_auth):
    users, pagination_item = server.users.get(req_options=req_opts)
    for user in users:
        pagination_item = server.users.populate_groups(user, req_options=req_opts)
        user_group.extend([empls[user.name] + (user.site_role, group.name) if user.name in empls.keys() else (None, None, None, user.name, user.site_role, group.name) for group in user.groups if group.name != "All Users"])
        
user_group = pd.DataFrame(user_group, columns=["hq", "dep", "job", "name", "site_role", "group"])
user_group = user_group.sort_values(by=["hq", "dep", "job", "name", "site_role", "group"])
user_group = user_group.reset_index(drop=True)

In [30]:
# user_group.to_csv("D:/디지털혁신팀/태블로/user_group.csv", index=False, encoding="euc-kr")
user_group.head()

,hq,dep,job,name,site_role,group
0,개발영업본부,개발부문,부문장,김태수,Viewer,본사팀장/그룹장
1,개발영업본부,개발영업본부,본부장,박희윤,Viewer,본부장 이상
2,개발영업본부,견적그룹(개발영업),그룹장,소문석,Viewer,본사팀장/그룹장
3,개발영업본부,도시개발1팀,팀원,임유빈,Viewer,"""본부장 회의"" 접근 가능 그룹"
4,개발영업본부,도시개발1팀,팀장,이현우,Viewer,"""분양일보"" 접근 가능 그룹"


### users.add()

In [115]:
with server.auth.sign_in(tableau_auth):
    name = "1101296"
    site_role = "Viewer"
    new_user = TSC.UserItem(name, site_role)
    
    new_user = server.users.add(new_user)   
    new_user.fullname = "황승환(디지털플랫폼팀)"
    new_user.email = "hwangsh@hdc-dvp.com"
    
    new_user = server.users.update(new_user, password=f"hdc{name}")

### user.remove()

In [114]:
with server.auth.sign_in(tableau_auth):
    server.users.remove("7103bcd9-9df5-4b0c-9e63-50698dd74392")

## Upload a Workbook

In [50]:
# filepath = "D:/디지털혁신팀/태블로/마이호미/마이호미_FINAL/마이호미.twbx"
# with server.auth.sign_in(tableau_auth):
#     wb = TSC.WorkbookItem(name="마이호미", project_id="44546321-94c8-4746-b0e5-0dfb3fa4ce5a")
#     wb = server.workbooks.publish(wb, filepath, "Overwrite")

### Sign Out

In [33]:
# server.auth.sign_out()